In [11]:
# Install Spark and Java

# !pip install pyspark
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# !tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
# !pip install -q findspark

pratciing builtin pyspark funtions

In [8]:
import os


# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
import findspark
findspark.init()

In [9]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameFunctions").getOrCreate()

In [52]:
from pyspark.sql.types import StructField, IntegerType, StructType

In [55]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)
df = df.withColumn("points", df["points"].cast(IntegerType()))
df = df.withColumn("price", df["price"].cast(IntegerType()))
# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [56]:
# Order a DataFrame by ascending values
df.orderBy(df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+--------------------+------------+------------------+---------------+
|country|         description|         designation|points|price|  province|            region_1|    region_2|           variety|         winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+------------+------------------+---------------+
|     US|Tasted in a fligh...|                null|   100|  200|California|         Napa Valley|        Napa|Cabernet Sauvignon|      Cardinale|
|     US|In a stunning lin...|En Chamberlin Vin...|   100|   65|    Oregon|Walla Walla Valle...|Oregon Other|             Syrah|         Cayuse|
|     US|A perfect score h...|     Hillside Select|   100|  215|California| Stags Leap District|        Napa|Cabernet Sauvignon|         Shafer|
|     US|A spectacular Pin...|Litton Estate Vin...|   100|  100|California|Russian River Valley|      Sonoma|        Pinot Noir|Wi

In [44]:
from pyspark.sql.functions import avg, count
df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [15]:
# Filter
df.filter("price<30").show(5)
# Filter by price on certain columns
df.filter("price<50").select(['points','country', 'winery','price']).show(5)

# Filter on exact state
df.filter(df["country"] == "US").show()

+--------+--------------------+--------------------+------+-----+------------+--------------------+--------+---------------+----------------+
| country|         description|         designation|points|price|    province|            region_1|region_2|        variety|          winery|
+--------+--------------------+--------------------+------+-----+------------+--------------------+--------+---------------+----------------+
|      US|Heitz has made th...|          Grignolino|    95|   24|  California|         Napa Valley|    Napa|           Ros̩|           Heitz|
|Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|    Bulgaria|                null|    null|         Mavrud|    Villa Melnik|
|  France|Pale in color, th...|Nonpareil Tr̩sor ...|    90|   22|France Other|        Vin Mousseux|    null|Sparkling Blend|  Bouvet-Ladubay|
|   Italy|Aromas of forest ...|                null|    90|   29|     Tuscany|Vino Nobile di Mo...|    null|     Sangiovese|      Avignonesi|
|   It

In [22]:
# Filter multiple conditions 
df.filter((df["country"] != "US") & (df['price'] < 20)).show(5)

+--------+--------------------+--------------------+------+-----+---------------+-------------+--------+--------------+--------------------+
| country|         description|         designation|points|price|       province|     region_1|region_2|       variety|              winery|
+--------+--------------------+--------------------+------+-----+---------------+-------------+--------+--------------+--------------------+
|Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|       Bulgaria|         null|    null|        Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|              Amandi|    90|   17|        Galicia|Ribeira Sacra|    null|       Menc�_a|      Don Bernardino|
|Portugal|It is the ripe da...|             Premium|    91|   15|       Alentejo|         null|    null|Portuguese Red|Adega Cooperativa...|
|Portugal|The local Baga gr...|Marqu̻s de Marial...|    86|   12|Beira Atlantico|         null|    null|          Baga| Adega de Cantanhede|
|Portugal|Thi

In [28]:
#what non us winerys have the wine over $500
df.filter((df["country"] != "US") & (df['price'] > 500)).select("winery").show()

+--------------------+
|              winery|
+--------------------+
|  Ch̢teau Haut-Brion|
|Ch̢teau La Missio...|
|        Marco Abella|
|        Louis Latour|
|        Louis Latour|
|            Penfolds|
|            Penfolds|
|      Emmerich Knoll|
|               Kopke|
|      Joseph Drouhin|
|               Kopke|
|         Robert Weil|
|        Royal Tokaji|
|  Ch̢teau Haut-Brion|
|                Krug|
|         Robert Weil|
|            Penfolds|
|            Penfolds|
|         Robert Weil|
|             Masseto|
+--------------------+
only showing top 20 rows



In [58]:
# avergage points and price per country
avg_df = df.groupBy("country").avg()
avg_df.select("country", "avg(points)", "avg(points)").show()

+--------------------+-----------------+-----------------+
|             country|      avg(points)|      avg(points)|
+--------------------+-----------------+-----------------+
|              Turkey|88.09615384615384|88.09615384615384|
|influence. This i...|             88.0|             88.0|
|             Germany|88.63183673469388|88.63183673469388|
|              France|88.92595050725325|88.92595050725325|
|              Greece|86.11764705882354|86.11764705882354|
|steak. It has fir...|             86.0|             86.0|
|            Slovakia|83.66666666666667|83.66666666666667|
|                null|             84.6|             84.6|
|           Argentina| 85.9960930562955| 85.9960930562955|
|             Albania|             88.0|             88.0|
|               India|           87.625|           87.625|
|               China|             82.0|             82.0|
|               Chile|86.29676753782668|86.29676753782668|
|             Croatia|86.28089887640449|86.2808988764044

In [65]:
# what non US country has the best cheap wine
cheap_wine = df.filter((df["country"] != "US") & (df['price'] < 20)).groupBy("country").avg().select("country", "avg(points)", "avg(price)")
cheap_wine.orderBy(cheap_wine["avg(points)"].desc()).show()

+--------------------+-----------------+------------------+
|             country|      avg(points)|        avg(price)|
+--------------------+-----------------+------------------+
|               India|             89.5|11.833333333333334|
|             Morocco|87.77777777777777| 15.11111111111111|
|              Turkey|87.65384615384616|15.692307692307692|
|             Austria|87.60230179028133|15.514066496163682|
|              Serbia|             87.0|              16.5|
|            Portugal|86.85879917184265|12.269979296066252|
|            Slovenia|            86.85|             14.05|
|             Germany| 86.7144754316069|14.702523240371846|
|         New Zealand|86.67769176636172|15.450387051372273|
|              Canada|            86.52|             15.14|
|               Italy| 86.2951883507439|14.614751503640392|
|              France|86.29309648768671|14.085991118288252|
|        South Africa|86.22628827483196|13.091112770724422|
|           Australia|86.18044444444445|